In [ ]:
from dask.distributed import LocalCluster, Client
from dask import delayed, compute

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

import pandas as pd
import numpy as np

In [ ]:
client = Client()

In [ ]:
def make_rand(max_val: float=1e2, min_val: float=1e3, size: int=10) -> np.ndarray:
    arr = np.arange(1e2) + 1e3
    np.random.shuffle(arr)
    return arr[:10].astype(int)

In [ ]:
def make_data(size: float=1e3) -> pd.DataFrame:
    size = int(size)
    a, b = make_rand().astype(str), make_rand().astype(str)
    df = pd.DataFrame(dict(
        a=np.random.choice(a, size=size),
        b=np.random.choice(b, size=size),
        c=np.random.randint(1e3, size=size)
    ), columns=('a', 'b', 'c'))
    for label in ['a', 'b']:
        df.loc[:, label] = df.loc[:, label].astype('category')
    return df

In [ ]:
data = make_data()

In [ ]:
for key, group in data.groupby(['a', 'b']):
    print(key, len(group))

In [ ]:
data.groupby(['a', 'b']).count()

In [ ]:
ddata = dd.from_pandas(data, npartitions=8)
ddata = client.persist(ddata)

In [ ]:
def func(df):
    return df.c.tolist()

In [ ]:
y = ddata.groupby(['a', 'b']).apply(func, meta=1)

In [ ]:
yy = y.compute()

In [ ]:
yy.head()

In [ ]:
msk = (ddata.a == '1017') & (ddata.b == '1031')
ddata.loc[msk].compute()